In [ ]:
import os
from glob import glob
import json
from tqdm import tqdm
import os 
import pandas as pd 
from glob import glob
from tqdm.auto import tqdm
tqdm.pandas()

def create_dir(base,ext):
    '''
        creates a directory extending base
        args:
            base    =   base path 
            ext     =   the folder to create
    '''
    _path=os.path.join(base,ext)
    if not os.path.exists(_path):
        os.mkdir(_path)
    return _path


save_path="/backup/RAW/DET/"
save_path=create_dir(save_path,"boise_state")
img_dir=create_dir(save_path,"images")
gt_dir=create_dir(save_path,"gts")


In [ ]:



def extract_info(_dir,coords,fmt):
    '''
        extracts information from boise-state annotations
    '''
    img_paths=[img_path for img_path in glob(os.path.join(_dir,f"*.{fmt}"))]
    liness=[]
    words=[]
    comps=[]
    chars=[]
    xmins=[]
    ymins=[]
    xmaxs=[]
    ymaxs=[]
    _paths=[]
    # images
    for img_path in tqdm(img_paths):
        base=img_path.split(".")[0]
        # text path
        _iden=os.path.basename(img_path).split(".")[0]
        text_path=os.path.join(_dir,coords,f"{_iden}.txt")
        with open(text_path,"r") as tf:
            lines=tf.readlines()
        for line in lines:
            parts=line.split()
            if len(parts)>4:
                line_num=parts[0].replace("\ufeff","")
                word_num=parts[1]
                label=parts[2]
                data=parts[3]
                x,y,w,h=[int(i) for i in parts[-1].split(",")]
                liness.append(line_num)
                words.append(word_num)
                chars.append(label)
                xmins.append(x)
                ymins.append(y)
                xmaxs.append(x+w)
                ymaxs.append(y+h)
                _paths.append(img_path)
                comps.append(data)
    df=pd.DataFrame({"line":liness,
                     "word":words,
                     "char":chars,
                     "comp":comps,
                     "xmin":xmins,
                     "ymin":ymins,
                     "xmax":xmaxs,
                     "ymax":ymaxs,
                     "image":_paths})
    return df

def check_missing(_dir,coords,fmt):
    '''
        checks for missing data
    '''
    img_paths=[img_path for img_path in glob(os.path.join(_dir,f"*.{fmt}"))]
    txt_paths=[txt_path for txt_path in glob(os.path.join(_dir,coords,"*.txt"))]
    # error check
    for img_path in tqdm(img_paths):
        if "jpg" in img_path:
            _iden=os.path.basename(img_path).split(".")[0]
            txt_path=os.path.join(_dir,coords,f"{_iden}.txt")
            if not os.path.exists(txt_path):
                print(img_path)
                for txt in txt_paths:
                    if _iden in txt:
                        print(txt)
                        niden=os.path.basename(txt).split('.')[0]
                        print(f"RENAME:{_iden} to {niden}")
                        os.rename(os.path.join(_dir,f"{_iden}.{fmt}"),
                                  os.path.join(_dir,f"{niden}.{fmt}"))
                        
                        



In [ ]:
readme_txt_path="/backup/RAW/BanglaC/README.txt"
base_path=os.path.dirname(readme_txt_path)
print(base_path)
assert len(os.listdir(base_path))==5,"WRONG PATH FOR README.txt"

os.listdir(base_path)
dfs=[]
# ## 1.Camera
_dir=os.path.join(base_path,'1. Camera','1. Essay')
coords='Character Coordinates_a'
fmt="jpg"
check_missing(_dir,coords,fmt)
dfs.append(extract_info(_dir,coords,fmt))
# ## 2. Scan
_dir=os.path.join(base_path,'2. Scan','1. Essay')
coords='Character Coordinates_a'
fmt="tif"
check_missing(_dir,coords,fmt)
dfs.append(extract_info(_dir,coords,fmt))
# # 3. Conjunct
_dir=os.path.join(base_path,'3. Conjunct')
coords='Character Coordinates'
fmt="tif"
check_missing(_dir,coords,fmt)
dfs.append(extract_info(_dir,coords,fmt))
df=pd.concat(dfs,ignore_index=True)
df   


In [ ]:
import shutil
import cv2
for img_path in tqdm(df.image.unique()):
    img=cv2.imread(img_path)
    base_=os.path.basename(img_path).split(".")[0]
    img_dst=os.path.join(img_dir,f"{base_}.jpg")
    cv2.imwrite(img_dst,img)
    
    idf=df.loc[df.image==img_path]

    box_text_list = []
    for line in idf.line.unique():
        linedf=idf.loc[idf.line==line]
        for word in linedf.word.unique():
            wdf=linedf.loc[linedf.word==word]
            # word
            x1=int(min(wdf.xmin.tolist()))
            x2=int(max(wdf.xmax.tolist()))

            y1=int(min(wdf.ymin.tolist()))
            y2=int(max(wdf.ymax.tolist()))

            label="".join(wdf.comp.tolist())
            box=[x1,y1,x2,y1,x2,y2,x1,y2]
            if label=="*":
                text = "###"
            else:
                text=label

            box_text_list.append(','.join(map(str, box)))
            box_text_list.append(',')
            box_text_list.append(text)
            box_text_list.append('\n')
    gts_text_file=os.path.join(gt_dir,f"{base_}.txt")
    with  open(gts_text_file, "w") as file:
        makeitastring = ''.join(map(str, box_text_list))
        file.write(makeitastring)
        file.close()    
            